In [117]:
import json
import os
import pandas as pd
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

from tqdm.notebook import  tqdm

with open('config.txt', 'r') as f:
    names = f.read().split('\n')

def process_roster(path):
    roster = pd.read_csv(path)
    roster['Game'] = roster.Game.str.split('@').apply(lambda x: set([y.strip() for y in x]))
    roster['Name'] = roster.Name
    roster['TeamAbbrev'] = roster.TeamAbbrev.apply(lambda x: set([x]))
    roster['OppAbbrev'] = roster.Game - roster.TeamAbbrev
    roster['Team'] = roster.TeamAbbrev.apply(lambda x: list(x)[0])
    roster['Opp'] = roster.OppAbbrev.apply(lambda x: list(x)[0])
    roster.Time = pd.to_datetime(roster.Time)
    roster.drop(['TeamAbbrev','OppAbbrev', 'Game'], axis = 1, inplace=True)
    return roster[['Name','Position', 'ID', 'Time', 'Salary', 'Team', 'Opp']]

import requests

def url_join(*args):
    return '/'.join(args)

class draftProcessor:
    def __init__(self, draft, load_ids = True):
        self.local_dir = 'data/NBA_API'
        self.base_url = "https://api-nba-v1.p.rapidapi.com"
        self.headers = {
            'x-rapidapi-host': "api-nba-v1.p.rapidapi.com",
            'x-rapidapi-key': "3c3966d785msh9c9c037d21ff8a9p1af469jsn475bf58bcbe1"
        }

        self.load_ids = load_ids
        self.draft = draft
        self.id_mapping = json.load(open(url_join(self.local_dir, 'id_mapping.json'), 'r'))

        if self.load_ids:
            self.get_player_ids()

        self.draft.index = [x.lower() for x in self.draft.index]
        self.draft = self.draft.join(pd.DataFrame.from_dict(self.id_mapping, orient='index').rename({0: 'api_id'}, axis = 1))
        self.draft.dropna(inplace=True)
        self.draft['api_id'] = self.draft['api_id'].astype(int)


    def get_player_ids(self):
        names = list(self.draft.index)

        for name in names:
            if name.lower() not in self.id_mapping:
                last_name = name.split(' ')[-1]
                print(f'id not found for {name}, getting id for {last_name}')
                try:
                    players = self.get_players({'name': last_name})
                except Exception as e:
                    print(f'failed. Error Message: {e}')
                    continue

                if type(players) == dict:
                    player_name = players['firstname'] + ' ' + players['lastname']
                    self.id_mapping[players[player_name.lower()]] = players['id']
                else:
                    for player in players:
                        player_name = player['firstname'] + ' ' + player['lastname']
                        player_id = player['id']
                        self.id_mapping[player_name.lower()] = player_id

                with open(url_join(self.local_dir, 'id_mapping.json'), 'w') as f:
                    json.dump(self.id_mapping,f)
            else:
                continue

    def get_players(self, params):
        url = url_join(self.base_url, 'players')
        print(url)
        resp= requests.get(url, headers = self.headers, params = params).json()
        return resp['response']


    def load_player_stats(self, params, ret = False):
        url = url_join(self.base_url, 'players','statistics')
        data_path = url_join(self.local_dir, 'stats', str(params['season']))
        params['id'] = int(params['id'])

        if f"{params['id']}.csv" in os.listdir(data_path):
            if ret:
                data = pd.read_csv(url_join(data_path, f"{params['id']}.csv"))
                data['id'] = params['id']
            else:
                return
        else:
            print(f"downloading data for id: {params['id']}")
            try:
                data = requests.get(url, headers= self.headers, params = params).json()
                data = pd.DataFrame(data['response']).drop(['team', 'player', 'game'], axis=1)
                data.to_csv(url_join(data_path, f"{params['id']}.csv"), index = False)
            except Exception as e:
                print(f"failed. {data} {key}")
                return

        if ret:
            return data


In [118]:
with open('config.txt', 'r') as f:
    names = f.read().split('\n')

name = names[0]
date = dt.datetime.now().strftime('%Y-%m-%d')
path = f'data/{name}/{date}.csv'


df = process_roster(path).set_index('Name')
df.head()

,Position,ID,Time,Salary,Team,Opp
Name,,,,,,
Giannis Antetokounmpo,PF/C,739957,2022-10-12 00:00:00+00:00,9900.0,MIL,CHI
Damian Lillard,PG,463121,2022-10-12 02:00:00+00:00,9600.0,POR,GSW
Ja Morant,PG,1065906,2022-10-11 23:00:00+00:00,9500.0,MEM,ORL
Shai Gilgeous-Alexander,PG,1067856,2022-10-11 23:00:00+00:00,9400.0,OKC,DET
Stephen Curry,PG,338365,2022-10-12 02:00:00+00:00,9300.0,GSW,POR


In [119]:
test = draftProcessor(df, False)

for name in test.id_mapping:
    print(name)
    test.load_player_stats({'id': test.id_mapping[name], 'season': 2021})

paolo banchero
downloading data for id: 3414


NameError: name 'key' is not defined

In [108]:
import numpy as np
df_factors = []

def calc_fp(scores):
    multiplier = np.array([1, .5, 1.25, 1.5, 2, 2, -.5, 1.5, 3])
    return np.dot(multiplier, scores)

for id in tqdm(test.draft.api_id):
    try:
        data = pd.read_csv(f'data/NBA_API/stats/2021/{id}.csv')
        data['min'] = data['min'].astype(str)
    except:
        continue

    data = data.drop(['pos', 'comment'], axis=1).dropna()
    data = data[data['min'] != "0"]

    if data.shape[0] <= 5:
        continue

    data['min'] = data['min'].str.split(':').apply(lambda x: float(f"{x[0]}.{str((int(x[1])/60)).split('.')[-1]}"))

    factors = pd.DataFrame(data.mean().rename(id)).T.reset_index()
    factors = factors.rename({'index' : 'api_id'}, axis = 1)
    factors['dd'] = ((data[['points', 'totReb', 'assists', 'steals', 'blocks']] > 10).sum(axis=1) >= 2).astype(int).mean()
    factors['td'] = ((data[['points', 'totReb', 'assists', 'steals', 'blocks']] > 10).sum(axis=1) >= 3).astype(int).mean()

    fp_cols = ['points', 'tpm', 'totReb', 'assists', 'steals', 'blocks', 'turnovers', 'dd', 'td']
    factors['fp'] = calc_fp(np.array(factors[fp_cols]).reshape(-1, 1))

    df_factors.append(factors)
    # test.load_player_stats(params = {'id': id, 'season': 2021})
df_factors = pd.concat(df_factors)
df_factors = pd.merge(test.draft.reset_index(), df_factors, left_on = 'api_id', right_on = 'api_id').rename({'index': 'Name'}, axis = 1)

  0%|          | 0/179 [00:00<?, ?it/s]

In [111]:
df_opt = df_factors[['Name', 'Position', 'ID', 'Team', 'Salary', 'fp']]
df_opt.columns = ['Name', 'Position', 'ID', 'TeamAbbrev', 'Salary', 'AvgPointsPerGame']
df_opt.to_csv('test_opt.csv')

df_opt.head()

,Name,Position,ID,TeamAbbrev,Salary,AvgPointsPerGame
0,giannis antetokounmpo,PF/C,739957,MIL,9900.0,57.912651
1,damian lillard,PG,463121,POR,9600.0,41.064516
2,ja morant,PG,1065906,MEM,9500.0,46.806338
3,stephen curry,PG,338365,GSW,9300.0,45.383152
4,cade cunningham,PG,1232182,DET,9200.0,35.625000


In [114]:
from pydfs_lineup_optimizer import get_optimizer, Site, Sport
optimizer = get_optimizer(Site.DRAFTKINGS, Sport.BASKETBALL)
optimizer.load_players_from_csv('test_opt.csv')

In [115]:
for lineup in optimizer.optimize(n=10):
    print(lineup)
    print(lineup.players)  # list of players
    print(lineup.fantasy_points_projection)
    print(lineup.salary_costs)

 1. PG      cole anthony                  PG    ORL                     32.451         5600.0$   
 2. SG      jalen suggs                   PG/SG ORL                     23.566         4400.0$   
 3. SF      andrew wiggins                SF    GSW                     30.062         5000.0$   
 4. PF      giannis antetokounmpo         C/PF  MIL                     57.913         9900.0$   
 5. C       jakob poeltl                  C     SAS                     32.736         5300.0$   
 6. G       ja morant                     PG    MEM                     46.806         9500.0$   
 7. F       jerami grant                  PF    POR                     31.647         6300.0$   
 8. UTIL    lonzo ball                    PG    CHI                     33.231         4000.0$   

Fantasy Points 288.41
Salary 50000.00

(cole anthony PG (ORL), jalen suggs PG/SG (ORL), andrew wiggins SF (GSW), giannis antetokounmpo C/PF (MIL), jakob poeltl C (SAS), ja morant PG (MEM), jerami grant PF (POR), lon